In [385]:
# Import dependencies
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from pathlib import Path
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
from src.get_project_data import housing_data, locale_data
import plotly.express as px



Create a Dataframe of average housing prices by zipcode

In [386]:

#Define
housing_df = housing_data("Austin")

#The path to our CSV file for Austin housing data
housing_df = housing_df[["date", "zipcode", "zip_name", "average_listing_price"]]


# Display Texas housing data in a table
housing_df.head()




,date,zipcode,zip_name,average_listing_price
15,2023-03-01,78652,"manchaca, tx",784162
59,2023-03-01,78662,"red rock, tx",856983
83,2023-03-01,78759,"austin, tx",698303
89,2023-03-01,78747,"austin, tx",493692
97,2023-03-01,78659,"paige, tx",688996


In [387]:
#Define
locale_df = locale_data()
locale_df["LOCALE"] = locale_df["LOCALE"].map({11: 'Urban', 12: 'Urban', 13: 'Urban',
                                 21: 'Suburban', 22: 'Suburban', 23: 'Suburban',
                                 31: 'Town', 32: 'Town', 33: 'Town',
                                 41: 'Rural', 42: 'Rural', 43: 'Rural'})



# Display Texas housing data in a table
locale_df.head()

,zipcode,LOCALE
26362,75001,Suburban
26363,75002,Suburban
26364,75006,Suburban
26365,75007,Suburban
26366,75009,Rural


In [388]:
# Merge the two dataframes using an inner join
df_full = housing_df.merge(locale_df, how='left', on="zipcode")
df_full = df_full [["date", "zipcode", "zip_name", "average_listing_price", "LOCALE"]]

#Make a new column for the year from the date column
df_full["year"] = pd.DatetimeIndex(df_full["date"]).year

#Replace date column with year column
df_full = df_full [["year", "zipcode", "zip_name", "average_listing_price", "LOCALE"]]

#Group by year 
df_fullgroup = df_full.groupby(["year", "zipcode"]).agg({"average_listing_price": "mean", "LOCALE": "first"})


# Display the data table
df_fullgroup



average_listing_price LOCALE
year zipcode                              
2016 76511            135496.833333  Rural
     76527            577072.166667  Rural
     76530                 265021.0  Rural
     76537            229168.666667  Rural
     76574            281678.166667  Rural
...                             ...    ...
2023 78757            892620.666667  Urban
     78758            471727.666667  Urban
     78759            707763.666667  Urban
     78953            498333.333333  Rural
     78957            514564.333333  Rural

[737 rows x 2 columns]

In [389]:
# Create a new dataframe for the average listing price by year only
df_fullgroup = df_fullgroup.reset_index()
df_fullgroup = df_fullgroup[["year", "average_listing_price"]]
df_fullgroup = df_fullgroup.groupby(["year"]).agg({"average_listing_price": "mean"})
df_fullgroup = df_fullgroup.reset_index()

#Add column for the year over year change
df_fullgroup["YOY"] = df_fullgroup["average_listing_price"].pct_change()



# Display the data table
df_fullgroup



,year,average_listing_price,YOY
0,2016,522215.078755,<NA>
1,2017,521456.947757,-0.001452
2,2018,534342.730653,0.024711
3,2019,573144.284236,0.072615
4,2020,611179.433947,0.066362
5,2021,767373.537975,0.255562
6,2022,912159.162088,0.188677
7,2023,891685.288889,-0.022446


Average Listing Prices in the Austin Area 2016-2023

In [390]:
#create a line graph for the average listing price by year using plotly express
fig = px.line(df_fullgroup, x="year", y="average_listing_price", title="Average Listing Price in Austin Area by Year")
fig.add_vline(x=2020, line_width=3, line_dash="dash", line_color="black")
fig.show()





In [ ]:
#Create a bar graph for percent change in average listing price by year
fig = px.bar(df_fullgroup, x="year", y="YOY", title="Percent Change in Average Listing Price in Austin Area by Year")
fig.add_vline(x=2020, line_width=3, line_dash="dash", line_color="black")
fig.show()


Average Listing Prices in the Austin Area by Locale 2016-2023

In [ ]:
#create dataframe for the average listing price per locale
df_fullgroup_locale = df_full.groupby(["year", "LOCALE"]).agg({"average_listing_price": "mean"})
df_fullgroup_locale = df_fullgroup_locale.reset_index()
df_fullgroup_locale = df_fullgroup_locale[["year", "LOCALE", "average_listing_price"]]
df_fullgroup_locale = df_fullgroup_locale.groupby(["year", "LOCALE"]).agg({"average_listing_price": "mean"})
df_fullgroup_locale = df_fullgroup_locale.reset_index()


# Display the data table
df_fullgroup_locale


,year,LOCALE,average_listing_price
0,2016,Rural,449859.342342
1,2016,Suburban,1083534.190476
2,2016,Urban,528225.660714
3,2017,Rural,462612.077156
4,2017,Suburban,985623.666667
5,2017,Urban,547284.395833
6,2018,Rural,462858.879336
7,2018,Suburban,981306.940476
8,2018,Urban,577678.375
9,2019,Rural,468798.803303


In [ ]:
#Add column for the year over year change
df_fullgroup_locale["YOY"] = df_fullgroup_locale["average_listing_price"].pct_change()

# Display the data table
df_fullgroup_locale



,year,LOCALE,average_listing_price,YOY
0,2016,Rural,449859.342342,<NA>
1,2016,Suburban,1083534.190476,1.408607
2,2016,Urban,528225.660714,-0.512497
3,2017,Rural,462612.077156,-0.124215
4,2017,Suburban,985623.666667,1.130562
5,2017,Urban,547284.395833,-0.444733
6,2018,Rural,462858.879336,-0.154263
7,2018,Suburban,981306.940476,1.1201
8,2018,Urban,577678.375,-0.411317
9,2019,Rural,468798.803303,-0.188478


In [ ]:
#Create plot graph for the average listing price by year and locale
fig = px.line(df_fullgroup_locale, x="year", y="average_listing_price", color="LOCALE", title="Average Listing Price in Austin Area by Year and Locale")
fig.add_vline(x=2020, line_width=2, line_dash="dash", line_color="black")
fig.show()



In [ ]:
#Calculate the percentage change in average listing price from 2016 to 2023 by locale
df_fullgroup_locale = df_fullgroup_locale[["year", "LOCALE", "YOY"]]
df_fullgroup_locale = df_fullgroup_locale[df_fullgroup_locale["year"] == 2023]
df_fullgroup_locale = df_fullgroup_locale[["LOCALE", "YOY"]]
df_fullgroup_locale = df_fullgroup_locale.set_index("LOCALE")
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"YOY": "Percent Change"})
df_fullgroup_locale = df_fullgroup_locale * 100
df_fullgroup_locale = df_fullgroup_locale.round(2)
df_fullgroup_locale = df_fullgroup_locale.sort_values("Percent Change", ascending=False)
df_fullgroup_locale = df_fullgroup_locale.reset_index()
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"LOCALE": "Locale"})
df_fullgroup_locale = df_fullgroup_locale.set_index("Locale")
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Percent Change": "Percent Change from 2016 to 2023"})
df_fullgroup_locale = df_fullgroup_locale.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup_locale = df_fullgroup_locale.reset_index()
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Locale": "Locale"})
df_fullgroup_locale = df_fullgroup_locale.set_index("Locale")
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Percent Change from 2016 to 2023": "Percent Change from 2016 to 2023"})
df_fullgroup_locale = df_fullgroup_locale.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup_locale = df_fullgroup_locale.reset_index()
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Locale": "Locale"})
df_fullgroup_locale = df_fullgroup_locale.set_index("Locale")
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Percent Change from 2016 to 2023": "Percent Change from 2016 to 2023"})
df_fullgroup_locale = df_fullgroup_locale.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup_locale = df_fullgroup_locale.reset_index()
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Locale": "Locale"})
df_fullgroup_locale = df_fullgroup_locale.set_index("Locale")
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Percent Change from 2016 to 2023": "Percent Change from 2016 to 2023"})
df_fullgroup_locale = df_fullgroup_locale.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup_locale = df_fullgroup_locale.reset_index()
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Locale": "Locale"})
df_fullgroup_locale = df_fullgroup_locale.set_index("Locale")
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Percent Change from 2016 to 2023": "Percent Change from 2016 to 2023"})
df_fullgroup_locale = df_fullgroup_locale.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup_locale = df_fullgroup_locale.reset_index()
df_fullgroup_locale = df_fullgroup_locale.rename(columns={"Locale": "Locale"})

# Display the data table
df_fullgroup_locale


,Locale,Percent Change from 2016 to 2023
0,Suburban,93.85
1,Rural,-11.64
2,Urban,-42.83


In [ ]:
#Calculate the percentage change in average listing price for df_fullgroup from 2016 to 2023 
df_fullgroup = df_fullgroup[["year", "YOY"]]
df_fullgroup = df_fullgroup[df_fullgroup["year"] == 2023]
df_fullgroup = df_fullgroup[["YOY"]]
df_fullgroup = df_fullgroup * 100
df_fullgroup = df_fullgroup.round(2)
df_fullgroup = df_fullgroup.rename(columns={"YOY": "Percent Change"})
df_fullgroup = df_fullgroup.reset_index()
df_fullgroup = df_fullgroup.rename(columns={"index": "Locale"})
df_fullgroup = df_fullgroup.set_index("Locale")
df_fullgroup = df_fullgroup.rename(columns={"Percent Change": "Percent Change from 2016 to 2023"})
df_fullgroup = df_fullgroup.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup = df_fullgroup.reset_index()
df_fullgroup = df_fullgroup.rename(columns={"Locale": "Locale"})
df_fullgroup = df_fullgroup.set_index("Locale")
df_fullgroup = df_fullgroup.rename(columns={"Percent Change from 2016 to 2023": "Percent Change from 2016 to 2023"})
df_fullgroup = df_fullgroup.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup = df_fullgroup.reset_index()
df_fullgroup = df_fullgroup.rename(columns={"Locale": "Locale"})
df_fullgroup = df_fullgroup.set_index("Locale")
df_fullgroup = df_fullgroup.rename(columns={"Percent Change from 2016 to 2023": "Percent Change from 2016 to 2023"})
df_fullgroup = df_fullgroup.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup = df_fullgroup.reset_index()
df_fullgroup = df_fullgroup.rename(columns={"Locale": "Locale"})
df_fullgroup = df_fullgroup.set_index("Locale")
df_fullgroup = df_fullgroup.rename(columns={"Percent Change from 2016 to 2023": "Percent Change from 2016 to 2023"})
df_fullgroup = df_fullgroup.sort_values("Percent Change from 2016 to 2023", ascending=False)
df_fullgroup = df_fullgroup.reset_index()
df_fullgroup = df_fullgroup.rename(columns={"Locale": "Locale"})
df_fullgroup = df_fullgroup.set_index("Locale")
df_fullgroup = df_fullgroup.rename(columns={"Percent Change from 2016 to 2023": "Percent Change from 2016 to 2023"})

# Display the data table
df_fullgroup


,Percent Change from 2016 to 2023
Locale,
7,-2.24
